In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier


##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [2]:
#Please change to your name.
name = "Michael Huang"

#Please use this to control EDA. 
show_eda = False

In [3]:
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
184,0,0.850,0.982,0.457,0.361,0.853,0.199,0.385,0.092,0.367,...,0.194,0.446,0.456,0.568,0.494,0.871,0.460,0.968,0.307,0.633
245,1,0.392,0.953,0.263,0.290,0.522,0.326,0.103,0.137,0.133,...,0.739,0.308,0.881,0.762,0.003,0.566,0.205,0.740,0.469,0.223
8,1,0.531,0.511,0.160,0.383,0.389,0.787,0.034,0.901,0.666,...,0.608,0.712,0.135,0.102,0.180,0.946,0.837,0.031,0.533,0.713
77,0,0.170,0.082,0.481,0.819,0.384,0.390,0.954,0.455,0.563,...,0.497,0.531,0.723,0.701,0.793,0.381,0.210,0.379,0.594,0.719
54,0,0.212,0.235,0.682,0.736,0.580,0.464,0.244,0.548,0.895,...,0.907,0.375,0.474,0.104,0.118,0.297,0.284,0.193,0.539,0.222


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [4]:
df.shape

(250, 201)

#### Do Modelling Stuff

Make a tree model (of some variety) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that should it occur. 

In [16]:
y = np.array(df["target"]).reshape(-1,1)
X = np.array(df.drop(columns={"target"}))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

pipe_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", RandomForestClassifier(random_state=0))
])

param_grid_rf = {
    "classifier__n_estimators": [100, 125],  
    "classifier__max_depth": [8, 9, 10],  
    "classifier__min_samples_leaf": [2,3],  
    "classifier__min_samples_split": [2, 3, 4, 5, 6, 7, 8, 9],  
    "classifier__max_features": ['sqrt', 'log2', 0.5],
    'classifier__criterion':["entropy"]
}



clf_rf = GridSearchCV(pipe_rf, param_grid=param_grid_rf, cv=5, n_jobs=-1)
clf_rf.fit(X_train, y_train.ravel())
best = clf_rf.best_estimator_

### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [13]:
print(best.score(X_test, y_test))
print(best)

0.7619047619047619
Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(criterion='entropy', max_depth=10,
                                        max_features='log2', min_samples_leaf=3,
                                        min_samples_split=9, n_estimators=125,
                                        random_state=0))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [18]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.6368744160417875
0.6364050632911392
Michael Huang 0.6366397396664634


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li> Tried random state numbers
<li> Tried different gridsearches
<li> Changed it to a forest instead of a tree
</ul>